In [ ]:
import json
# import pandas as pd
import requests
import time
import pymysql
import datetime
# from tqdm import tqdm
# import numpy as np

# index = np.arange(5)

# test_data = pd.DataFrame([{'name':'제주시청','address':'제주시 이도이동 1176-1'},
#                   {'name':'함덕해수욕장','address':'제주시 조천읍 함덕리 1004-10'},
#                   {'name':'협재해수욕장','address':'제주시 한림읍 협재리 2497-1'},
#                   {'name':'쇠소깍','address':'서귀포시 남원읍 하례리 995-1'},
#                   {'name':'성산일출봉','address':'서귀포시 성산읍 성산리 1'}],index=index)

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)
cursor = conn.cursor()


sql = "SELECT `c_addr`, `c_idx` FROM `hairshop` WHERE `c_idx` < 10001 AND `c_idx` > 180;"
# sql = "SELECT * FROM `hairshop` order by c_visitor_review DESC limit 1000;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
cursor.execute(sql)
result = cursor.fetchall()

param1 = 'service=address'
param2 = 'request=getCoord'
param3 = 'key=4A1CF381-0E54-33E4-AC30-0357FFAE3951'
param4 = 'type=road'
param5 = 'refine=true'
url = 'http://api.vworld.kr/req/address?{0}&{1}&{2}&{3}&{4}&address='.format(param1,param2,param3,param4,param5)

for res in result:
    link = url + res[0]
    page = requests.get(link)
    time.sleep(1.5)
    json_data = page.json()
#     print(json_data)
    status = json_data['response']['status']
    
    if status == 'OK':
        print(res[1] + ' ok')
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
#         print('status = {0}\n x = {1} \n y = {2}'.format(status, x, y))
    else :
        x = 0
        y = 0
    
    
    try:
        sql = "UPDATE hairshop SET c_lng=%s, c_lat=%s, edit_date=%s WHERE c_idx=%s"
        val = (x, y, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), res[1])
        cursor.execute(sql, val)
    except:
        print("sql-Update Hairshop fail 주소 : {0}".format(res[0]))
    conn.commit()
        
conn.close()

# add = 'http://api.vworld.kr/req/address?'

# param6 = 'address=부림로169번길 22'

# def request_geo(road):
#     link = url + res
#     page = requests.get(link)
#     time.sleep(0.5)
#     json_data = page.json()
    
#     status = json_data['response']['status']
#     x = json_data['response']['result']['point']['x']
#     y = json_data['response']['result']['point']['y']
#     print('status = {0}\n x = {1} \n y = {2}'.format(status, x, y))
    

# def extraction_geo(test_data):
#     geocode = pd.DataFrame(columns = ['name','address', 'x', 'y'])
#     none = None
#     for idx, road in tqdm(zip(test_data.index ,test_data['address'])):
#         name = str(test_data['name'][idx])
#         if len(str(road)) <= 5:
#             geocode = geocode.append(
#                     pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':none,
#                     'y':none},
#                     index=[idx]))
#             continue

#         json_data = request_geo(road)

#         if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR':
#             geocode = geocode.append(
#                     pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':none,
#                     'y':none},
#                     index=[idx]))
#             continue

#         x = json_data['response']['result']['point']['x']
#         y = json_data['response']['result']['point']['y']

#         geocode = geocode.append(
#             pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':float(x),
#                     'y':float(y)},
#                     index=[idx]))
#     return geocode

# result = extraction_geo(test_data)
